In [77]:
import random

In [78]:
def get_stats(CH):
    print(f'(R,G,B,A) = ({CH[0]},{CH[1]},{CH[2]},{CH[3]})')
    print(f'Roughness Factor = {CH[4]}')
    print(f'Number of vertices = {len(CH[5])}')
    print(f'Number of faces = {len(CH[6])}')

### Importing function for generating chromosome

In [131]:
from CGen import *

In [132]:
%%time
L,B,H=5,5,5

P1=[]

for _ in range(10):
    CH=generate_chromosome(L,B,H,random.random())
    P1.append(CH)
    get_stats(CH)
    print('-'*125)

(R,G,B,A) = (212,128,219,0.6446246782620595)
Roughness Factor = 0.08354043025433988
Number of vertices = 106
Number of faces = 2774
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (186,228,52,0.09265597361093103)
Roughness Factor = 0.7936586015680909
Number of vertices = 72
Number of faces = 1272
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (33,28,133,0.5863803725801583)
Roughness Factor = 0.20280905610423372
Number of vertices = 11
Number of faces = 32
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (43,47,171,0.4421072433292791)
Roughness Factor = 0.4357143042298688
Number of vertices = 78
Number of faces = 1491
---------------------------------------------------------------------------------------------------------

In [ ]:
len()

## Mutation

In [81]:
def mutate(C):
    for 

IndentationError: expected an indented block (Temp/ipykernel_28412/3924105588.py, line 2)